In [1]:
from autotst.reaction import *

In [3]:
rxn = AutoTST_Reaction("CC=C(C)C+[O]O_[CH2]C=C(C)C+OO", "H_Abstraction")

final_log_file = rxn.label + "_overall.log"

from ase.io.gaussian import *
optimized_ase_ts = read_gaussian_out(final_log_file)

rxn.ts.ase_ts = optimized_ase_ts
rxn.ts.update_from_ase_ts()


rmg.py:72 __init__ WARNING Should only make one instance of RMGDatabase because it's stored as a module-level variable!
rmg.py:73 __init__ WARNING Unexpected behaviour may result!
thermo.py:839 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:856 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-Py/../RMG-database/input/transport/groups...
database.py:167 loadFamilies INFO Loading the user-speci

In [5]:
from cclib.io import ccread
log_file_info = ccread(final_log_file)

rxn.vibrations = zip(log_file_info.vibfreqs, log_file_info.vibdisps)

got_imaginary_frequency = False
for vib, displacements in rxn.vibrations:
    if vib < 0: # Finding the imaginary frequency
        got_imaginary_frequency = True
        post_vibration_ts = rxn.ts.ase_ts.copy() 
        post_vibration_ts.arrays["positions"] -= displacements
        
assert got_imaginary_frequency == True, "Did not find an imaginary frequency"
def percent_change(original,new):
    percent_change = (abs(new - original) / original) * 100
    return percent_change

results = []

for torsion in rxn.ts.torsions:
    i,j,k,l = torsion.indices
    before = rxn.ts.ase_ts.get_dihedral(i,j,k,l)
    after = post_vibration_ts.get_dihedral(i,j,k,l)
    results.append(["Tor", torsion.indices, torsion.reaction_center, percent_change(before, after)])
    
for angle in rxn.ts.angles:
    i,j,k = angle.indices
    before = rxn.ts.ase_ts.get_angle(i,j,k)
    after = post_vibration_ts.get_angle(i,j,k)
    results.append(["Ang", angle.indices, angle.reaction_center, percent_change(before, after)])

for bond in rxn.ts.bonds:
    i,j = bond.indices
    before = rxn.ts.ase_ts.get_distance(i,j)
    after = post_vibration_ts.get_distance(i,j)
    results.append(["Bond", bond.indices, bond.reaction_center, percent_change(before, after)])

results = pd.DataFrame(results)
results.columns = ["type", "index", "center", "percent_change"]

if (np.log10(((results[results.center == "Yes"].mean()))) > np.log10(((results[results.center != "Yes"].mean()))) + 1).all():
    print "Vibrational analysis was successful"
    print True

else:
    print "Cannot reasonably say that we have arrived at a TS through vibrational analysis.\nRunning an IRC calc."
    print False

        

Vibrational analysis was successful
True


In [ ]:
from ase.io.gaussian import *
new_ase_ts = read_gaussian_out(log_file)

In [ ]:
rxn.ts.ase_ts = new_ase_ts
rxn.ts.update_from_ase_ts()


In [ ]:
rxn.ts.view_ts()

In [ ]:
import cclib
from cclib.io import ccread
from ase.visualize import view

In [ ]:
from cclib.io import ccread
log_file_info = ccread(log_file)
log_file_info

In [ ]:
imaginary_displacements = log_file_info.vibdisps[0]
imaginary_displacements

In [ ]:
copy_ts = rxn.ts.ase_ts.copy()
copy_ts

In [ ]:
copy_ts is rxn.ts.ase_ts

In [ ]:
copy_ts.arrays["positions"]

In [ ]:
copy_ts.arrays["positions"] -= imaginary_displacements
copy_ts.arrays["positions"]

In [ ]:
view(rxn.ts.ase_ts)

In [ ]:
view(copy_ts)

In [ ]:
def percent_change(original,new):
    percent_change = (abs(new - original) / original) * 100
    return percent_change

In [ ]:
results = []

In [ ]:
for combo in rxn.ts.torsions:
    i,j,k,l = combo.indices
    print combo.reaction_center
    before = rxn.ts.ase_ts.get_dihedral(i,j,k,l)
    after = copy_ts.get_dihedral(i,j,k,l)
    #print before, after
    
    results.append(["Tor", combo.indices, combo.reaction_center, percent_change(before, after)])
    print percent_change(before, after)
    print

In [ ]:
for combo in rxn.ts.angles:
    i,j,k = combo.indices
    print combo.reaction_center
    
    before = rxn.ts.ase_ts.get_angle(i,j,k)
    after = copy_ts.get_angle(i,j,k)
    
    #print int(before), int(after)

    results.append(["Ang", combo.indices, combo.reaction_center, percent_change(before, after)])
    print percent_change(before, after)
    print
    


In [ ]:
for combo in rxn.ts.bonds:
    i,j = combo.indices
    print combo.reaction_center
    
    before = rxn.ts.ase_ts.get_distance(i,j)
    after = copy_ts.get_distance(i,j)
    #print before, after
    results.append(["Bond", combo.indices, combo.reaction_center, percent_change(before, after)])
    
    print percent_change(before, after)
    print

In [ ]:
results = pd.DataFrame(results)
results.columns = ["type", "index", "center", "percent_change"]
results.sort_values("percent_change", ascending=False)

In [ ]:
b = results[results.center == "Yes"].mean()
print results[results.center == "No"].mean()
print results[results.center == "Close"].mean()

In [ ]:
if ((float(results[results.center == "Yes"].mean()) > float(results[results.center == "Close"].mean())) and
    float(results[results.center == "Close"].mean()) > float(results[results.center == "No"].mean())):
    print "Yay"

In [ ]:
if (np.log10(((results[results.center == "Yes"].mean()))) > np.log10(((results[results.center != "Yes"].mean()))) + 1).all():
    
    

In [ ]:
np.log10(((results[results.center != "Yes"].mean()))) + 1

In [ ]:
np.log10((float(results[results.center == "No"].mean()))) + 1